In [ ]:
import json
l1 = []
l2 = []
l3 = []

with open('/Users/mac/Desktop/python_project/scraped_data/1960-1969.txt','r') as f:
    lines = f.readlines()

    i = 0
    for line in lines:
        i += 1
        try:
            dic = json.loads(line,strict = False)
            docs = dic['response']['docs']
            for doc in docs:
                l1.append(doc["web_url"])
                l2.append(doc["pub_date"])
                l3.append(doc["headline"]["main"])
        except Exception as e:
            print(e) #print lines containing damaged data
            continue
print(len(lines))

In [ ]:
#remove NoneType in l3

def not_empty(s):
    return s and s.strip()
l3_remove = list(filter(not_empty,l3))



#remove punctuations

punctuation = ['(', ')', '?', ':', ';', ',', '.', '!', '/', '"', "'","-","%","$","&","*","+","0","1","2","3","4","5","6","7","8","9"]
l4 = []

for line in l3_remove:
    newline = ''
    for c in line:
        if c not in punctuation:
            newline = newline + c
    l4.append(newline)

#case conversion
l4_low = list(map(lambda x: x.lower(), l4))

In [ ]:
#Tokenization, Stemming, Lemmatization

from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
 
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

l4_tag = []
for headline in l4_low:
    token = word_tokenize(headline)  
    tagged_sent = pos_tag(token)     
    l4_tag.append(tagged_sent)
    
wnl = WordNetLemmatizer()
l4_lemmas = []

for tag in l4_tag:
    for t in tag:
        wordnet_pos = get_wordnet_pos(t[1]) or wordnet.NOUN
        l4_lemmas.append(wnl.lemmatize(t[0], pos=wordnet_pos)) 

In [ ]:
#remove stopwords

from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

l4_stop = []
for word in l4_lemmas: 
    if word not in en_stops:
        l4_stop.append(word)

In [ ]:
#get high-frequency words

l6 = []
for line in l4_stop:
    newline = line.split(' ')
    l6.extend(newline)

wordcount = {}
for word in l6:
    if word in wordcount:
        wordcount[word] += 1
    else:
        wordcount[word] = 1

sorted(wordcount.items(),key=lambda item:item[1],reverse = True)


In [ ]:
#calculate the mentioning possibility of each high-frequency word

l_all=sorted(wordcount.items(),key=lambda item:item[1],reverse = True)

words_all = []

for item in l_all:
    if not item[0] in words_all:
        words_all.append(item[0])

print(words_all)  

word_dic = {}

for word in words_all:
    word_dic[word] = l3.count(word)


word_dic_sorted = dict(sorted(word_dic.items(), key=lambda i:i[0]))

import pandas as pd
 
word_df = pd.DataFrame.from_dict(word_dic_sorted, orient='index')
 
word_df.to_excel('/Users/mac/Desktop/1960.xlsx',sheet_name='Sheet1')